In [1]:
# Generic: A Modular Multi-Pipeline Framework for Probability Fusion Ensembles
# Specific: Cross-Lingual Sentiment Analysis with Probability Fusion Ensembles: A Modular Multi-Pipeline Framework for Low-Resource Languages

import os
import pandas as pd
import pprint

from pathlib import Path
from transformers import pipeline

from src import sentiment, helper, utility, context
from src.config import *
from src.metrics import evaluate_pipe

os.environ["TOKENIZERS_PARALLELISM"] = "false"
helper.list_config()

mbert_context = context.setup_pipeline(MBertConfig, require_translation = False)
mbert_trainer = sentiment.train(mbert_context)

#mamba_context = context.setup_pipeline(MambaConfig, require_translation = True)
#mamba_trainer = sentiment.train(mamba_context)

# TBA: FUSION #

if AppConfig.INFER_SAMPLE:
    sample_texts = [
        "Maganda ang serbisyo at mabilis ang delivery!"
        "Sobrang pangit ng karanasan ko.",
        "It was okay, nothing special.",
    ]
    #sentiment.infer(sample_texts, mbert_trainer.tokenizer, mbert_trainer.model)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/69 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

[DBG-LIVE] epoch_start=0 global_step=0 wrapped=AcceleratedOptimizer base=AdamW lr=2e-05 id=136365644563424


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,Precision Macro,Recall Macro
1,No log,1.053314,0.300000,0.228571,0.206349,0.305556


[DBG-LIVE] epoch_start=1.0 global_step=9 wrapped=AcceleratedOptimizer base=AdamW lr=2e-05 id=136365644563424
Switching optimizer to SGD right after epoch 2 (next epoch will print SGD in your debug)


AttributeError: 'SwitchOptimizerCallback' object has no attribute 'lr'